# Feature search

We are looking for a feature that has the following properites:
* Predicted by presence of a single token with high precision and recall
* Minimal number of attention heads contributing to SAE feature

This is the plan:
* Let `DD := data/processed/gelu_1l_audit`
* Compute and store activations, logits, SAE features for all data points
    * Store in `DD/activations`
* Find (feature, token) pairs:
    * Take top K activations for each SAE feature
    * Take the top token in the dataset for these examples
    * Store in `DD/token_feature_pairs`
* Evaluate precision and recall of `FEATURE IF TOKEN` predictor
    * Store in `DD/precision_recall.tsv`

In [ ]:
# Load model, data, and SAE

